In [115]:
from math import exp
import sys
# Backpropagation for fully connected 2-layer feed-forward networks
class MLFFANN(object):
    
    def __init__(self, learning_rate = 0.5, n_epoch = 10):
        self.learning_rate = learning_rate
        self.n_epoch = n_epoch
        
    def initialize_network(self, n_inputs, n_hidden, n_outputs):
        network = []
        hidden_layer = [{'weights':[0 for i in range(n_inputs + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
        output_layer = [{'weights':[0 for i in range(n_hidden + 1)]} for i in range(n_outputs)]
        network.append(output_layer)
        self.ann = network
        return network
    
    def printNetWork(self):
        for i, layer in enumerate(self.ann):
            if i!=len(self.ann)-1:
                print("Hidden Layer: ",layer)
            else:
                print("Output Layer: ",layer)
    
    # forward propagation
    # Calculate neuron activities with respect to an input, calculated as the weighted sum of the inputs
    def sum_weighted_input(self, weights, inputs):
        # set init as b
        activities = weights[-1]
        # calculated the weighted sum of the inputs
        for i in range(len(inputs)):
            activities += weights[i] * inputs[i]
        return activities
    
    # activation function
    def sigmoid_activation_function(self, activities):
        return 1.0 / (1.0 + exp(-activities))
    
    # Forward propagate input to a network output
    def forward_propagate(self, network, row):
        inputs = row
        # go through the entire network
        for layer in network:
            next_layer_inputs = []
            # calculate each neurons acticity
            for neuron in layer:
                neuron_activity = self.sum_weighted_input(neuron['weights'], inputs)
                neuron['output'] = self.sigmoid_activation_function(neuron_activity)
                next_layer_inputs.append(neuron['output'])
            # use it as input for next layer
            inputs = next_layer_inputs
        return inputs
    
    # calculate back propagate error (backpropagating error is rooted in calculus)
    # Calculate the derivative of an neuron output
    def sigmoidPrime(self, output):
        #derivative of sigmoid function
        sigmoid_derivative = output * (1.0 - output)
        return sigmoid_derivative
    
    # Backpropagate error and store in neurons
    def backward_propagate_error(self, network, expected):
        for i in reversed(range(len(network))):
            layer = network[i]
            errors = []
            # process output layer (Label-Predict)
            if i == len(network)-1:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(expected[j] - neuron['output'])
            # process hidden layer
            else:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in network[i + 1]:
                        error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * self.sigmoidPrime(neuron['output'])
    
    # Update network weights with update parameter (delta error)
    def update_weights(self, network, row_data, l_rate):
        # loop through network
        for i in range(len(network)):
            # start with one sample row of data
            inputs = row_data
            # input of next layer is output of previous layer
            if i != 0:
                inputs = [neuron['output'] for neuron in network[i - 1]]
            # update weight of neural in layer i
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                # update bias
                neuron['weights'][-1] += l_rate * neuron['delta']
    
    # Train a network for a fixed number of epochs
    def train_network(self, train_data, train_label):
        l_rate = self.learning_rate
        n_epoch=self.n_epoch
        # init network
        n_inputs = len(train_data[0])
        n_outputs = len(set(train_label))
        self.initialize_network(n_inputs, 2, n_outputs)
        print("Init net: ")
        self.printNetWork()
        # start train
        for epoch in range(n_epoch):
            sum_error = 0
            for row_index, row in enumerate(train_data):
                outputs = self.forward_propagate(self.ann, row)
                #print(self.ann)
                expected = [0 for i in range(n_outputs)]
                expected[train_label[row_index]] = 1
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                self.backward_propagate_error(self.ann, expected)
                self.update_weights(self.ann, row, l_rate)
#                 print(self.ann)
#                 sys.exit("Test")
            # print(self.ann)
            print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [116]:
data = [[2.7810836,2.550537003],
    [1.465489372,2.362125076],
    [3.396561688,4.400293529],
    [1.38807019,1.850220317],
    [3.06407232,3.005305973],
    [7.627531214,2.759262235],
    [5.332441248,2.088626775],
    [6.922596716,1.77106367],
    [8.675418651,-0.242068655],
    [7.673756466,3.508563011]]
label = [0,0,0,0,0,1,1,1,1,1]
neural_net = MLFFANN(learning_rate = 0.5, n_epoch = 20)
neural_net.train_network(data, label)
neural_net.printNetWork()

Init net: 
Hidden Layer:  [{'weights': [0, 0, 0]}, {'weights': [0, 0, 0]}]
Output Layer:  [{'weights': [0, 0, 0]}, {'weights': [0, 0, 0]}]
>epoch=0, lrate=0.500, error=5.149
>epoch=1, lrate=0.500, error=5.146
>epoch=2, lrate=0.500, error=5.116
>epoch=3, lrate=0.500, error=5.058
>epoch=4, lrate=0.500, error=4.981
>epoch=5, lrate=0.500, error=4.887
>epoch=6, lrate=0.500, error=4.767
>epoch=7, lrate=0.500, error=4.609
>epoch=8, lrate=0.500, error=4.396
>epoch=9, lrate=0.500, error=4.123
>epoch=10, lrate=0.500, error=3.802
>epoch=11, lrate=0.500, error=3.456
>epoch=12, lrate=0.500, error=3.111
>epoch=13, lrate=0.500, error=2.785
>epoch=14, lrate=0.500, error=2.487
>epoch=15, lrate=0.500, error=2.220
>epoch=16, lrate=0.500, error=1.985
>epoch=17, lrate=0.500, error=1.779
>epoch=18, lrate=0.500, error=1.598
>epoch=19, lrate=0.500, error=1.442
Hidden Layer:  [{'weights': [-1.054627733312686, 1.2517601148488124, 0.39974451686533635], 'output': 0.04585974936790121, 'delta': -0.00737181379658567

In [ ]:
sys.exit("Test")

In [2]:
# Calculate the derivative of an neuron output
def sigmoidPrime(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        print(len(layer))
        errors = []
        # process output layer
        if i == len(network)-1:
            print("Output Layer: ",layer)
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
            print("Out error: ",errors)
        # process hidden layer
        else:
            for j in range(len(layer)):
                print("Hidden Layer: ",layer)
                error = 0.0
                for neuron in network[i + 1]:
                    print(neuron)
                    print(neuron['weights'][j])
                    print(neuron['delta'])
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
                print("Hidden error: ", errors)
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoidPrime(neuron['output'])
            print("neural",j, ": ",neuron)

# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
        [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

2
Output Layer:  [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]
Out error:  [-0.6213859615555266, 0.34263065440130236]
neural 0 :  {'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}
neural 1 :  {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}
1
Hidden Layer:  [{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}
0.2550690257394217
-0.14619064683582808
{'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}
0.4494910647887381
0.0771723774346327
Hidden error:  [-0.0026004117552590952]
neural 0 :  {'output': 0.71056688831

In [19]:
from math import exp
 
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(inputs)):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            print(neuron)
            new_inputs.append(neuron['output'])
        print(new_inputs)
        inputs = new_inputs
    return inputs

# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0]
output = forward_propagate(network, row)
print(output)

{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941}
[0.7105668883115941]
{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887}
{'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748}
[0.6629970129852887, 0.7253160725279748]
[0.6629970129852887, 0.7253160725279748]


In [4]:
import numpy as np

# X = (hours sleeping, hours studying), y = score on test
X = np.array(([2, 9], [1, 5], [3, 6]), dtype=float)
y = np.array(([92], [86], [89]), dtype=float)

# scale units
X = X/np.amax(X, axis=0) # maximum of X array
y = y/100 # max test score is 100

class Neural_Network(object):
    def __init__(self):
        #parameters
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3

        #weights
        self.W1 = np.random.randn(self.inputSize, self.hiddenSize) # (3x2) weight matrix from input to hidden layer
        self.W2 = np.random.randn(self.hiddenSize, self.outputSize) # (3x1) weight matrix from hidden to output layer

    def forward(self, X):
        #forward propagation through our network
        self.z = np.dot(X, self.W1) # dot product of X (input) and first set of 3x2 weights
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = np.dot(self.z2, self.W2) # dot product of hidden layer (z2) and second set of 3x1 weights
        o = self.sigmoid(self.z3) # final activation function
        return o 

    def sigmoid(self, s):
        # activation function |
        return 1/(1+np.exp(-s))

    def sigmoidPrime(self, s):
        #derivative of sigmoid
        return s * (1 - s)

    def backward(self, X, y, o):
        # backward propgate through the network
        self.o_error = y - o # error in output
        self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid to error

        self.z2_error = self.o_delta.dot(self.W2.T) # z2 error: how much our hidden layer weights contributed to output error
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.z2) # applying derivative of sigmoid to z2 error

        self.W1 += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
        self.W2 += self.z2.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
    
    def get_loss(self):
        return self.loss

    def train (self, X, y):
        o = self.forward(X)
        self.loss = np.mean(np.square(y - o))
        self.backward(X, y, o)
        
NN = Neural_Network()
for i in range(1000): # trains the NN 1,000 times
    print("Input: ", X)
    print("Actual Output: ", y)
    NN.train(X, y)
    print("Predicted Output: ", NN.forward(X))
    print("Loss: ", NN.get_loss()) # mean sum squared loss

Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.48355015]
 [0.49716298]
 [0.48366459]]
Loss:  0.22814108163664146
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.55398611]
 [0.56232882]
 [0.54091087]]
Loss:  0.16241587856049275
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.611416  ]
 [0.61516194]
 [0.58805023]]
Loss:  0.11481250604666433
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.65710029]
 [0.65696712]
 [0.62632811]]
Loss:  0.08211447551793338
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  

Predicted Output:  [[0.90859275]
 [0.88776349]
 [0.87468608]]
Loss:  0.00037892114091526963
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.9086062 ]
 [0.88776489]
 [0.87472119]]
Loss:  0.00037848435078967016
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90861898]
 [0.88776563]
 [0.87475549]]
Loss:  0.00037804992804971936
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90863111]
 [0.88776575]
 [0.87478899]]
Loss:  0.0003776177577452847
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90864263]
 [0.88776526]
 [0.87482173]]
Loss:  0.0003771877333141723
Input:  [[0.66666667 1.        ]
 [0.33333

Predicted Output:  [[0.90858781]
 [0.886447  ]
 [0.87673896]]
Loss:  0.0003355095727946042
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90858429]
 [0.8864329 ]
 [0.87675122]]
Loss:  0.0003351789346113729
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90858076]
 [0.88641882]
 [0.87676347]]
Loss:  0.00033484893772856884
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90857723]
 [0.88640474]
 [0.87677569]]
Loss:  0.0003345195806341088
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.9085737 ]
 [0.88639068]
 [0.87678788]]
Loss:  0.000334190861820582
Input:  [[0.66666667 1.        ]
 [0.33333333

Predicted Output:  [[0.90823378]
 [0.88504746]
 [0.87793947]]
Loss:  0.00030402937086480646
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90823066]
 [0.88503497]
 [0.87795022]]
Loss:  0.00030375853085855644
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90822754]
 [0.88502251]
 [0.87796096]]
Loss:  0.00030348819354706166
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90822443]
 [0.88501005]
 [0.87797169]]
Loss:  0.0003032183577829802
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90822133]
 [0.88499762]
 [0.87798241]]
Loss:  0.0003029490224219403
Input:  [[0.66666667 1.        ]
 [0.33333

Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90792215]
 [0.88377479]
 [0.87904819]]
Loss:  0.00027724136836741254
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90791953]
 [0.88376389]
 [0.8790578 ]]
Loss:  0.0002770191476992126
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90791693]
 [0.88375301]
 [0.8790674 ]]
Loss:  0.0002767973205091885
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90791433]
 [0.88374213]
 [0.87907699]]
Loss:  0.00027657588592949674
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted

Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90750028]
 [0.88195444]
 [0.88068067]]
Loss:  0.00024185872734721876
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90749832]
 [0.88194568]
 [0.88068867]]
Loss:  0.00024169681905443992
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90749636]
 [0.88193692]
 [0.88069666]]
Loss:  0.00024153517503204663
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.9074944 ]
 [0.88192817]
 [0.88070465]]
Loss:  0.0002413737947332684
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90749245]
 [0.88191943]
 [0.88071263]]
Loss:  0.00024121267761265154


Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90727906]
 [0.88094527]
 [0.88161001]]
Loss:  0.00022377265999729508
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90727742]
 [0.8809376 ]
 [0.88161714]]
Loss:  0.00022363952509617992
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90727577]
 [0.88092995]
 [0.88162426]]
Loss:  0.0002235065961581756
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90727413]
 [0.8809223 ]
 [0.88163137]]
Loss:  0.00022337387277550366
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicte

Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90706534]
 [0.87992612]
 [0.88256634]]
Loss:  0.00020664576063363682
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90706399]
 [0.8799195 ]
 [0.8825726 ]]
Loss:  0.00020653835458579934
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90706263]
 [0.87991289]
 [0.88257886]]
Loss:  0.00020643110478415916
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predicted Output:  [[0.90706128]
 [0.87990629]
 [0.88258512]]
Loss:  0.00020632401093490007
Input:  [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output:  [[0.92]
 [0.86]
 [0.89]]
Predict